<a href="https://colab.research.google.com/github/Danusyasundaravel/Deepfake-detection/blob/main/Architectural_Design_Generator_(ArchGen).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
adilmohammed_floor_plan_images_and_their_details_path = kagglehub.dataset_download('adilmohammed/floor-plan-images-and-their-details')

print('Data source import complete.')


In [ ]:
import cv2
import pandas as pd
import os
import matplotlib.pyplot as plt

# Path to the folder containing the images
images_folder = "/kaggle/input/floor-plan-images-and-their-details/images/images"

# Path to the CSV file containing details
dataset_path = "/kaggle/input/floor-plan-images-and-their-details/house_plans_details.csv"

# Load the CSV file into a DataFrame
data = pd.read_csv(dataset_path)

# Get the list of image filenames
images = os.listdir(images_folder)

# Display the first four images
plt.figure(figsize=(12, 6))
for i in range(3):
    # Read the image
    image_path = os.path.join(images_folder, images[i])
    image = cv2.imread(image_path)

    # Convert BGR image to RGB for displaying with matplotlib
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image
    plt.subplot(1, 4, i+1)
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Image {i+1}")

plt.show()
data.head(3)


In [ ]:
print("\nMissing values:")
print(data.isnull().sum())

# Convert data types of "Square Feet", "Beds", and "Garages" columns to integer
data["Square Feet"] = data["Square Feet"].astype(int)
data["Beds"] = data["Beds"].astype(int)
data["Garages"] = data["Garages"].astype(int)

# Check the data types after conversion
print("Data types after conversion:")
print(data.dtypes)

In [ ]:
import seaborn as sns
data1 = data.drop('Image Path', axis=1)
sns.heatmap(data1.corr(), annot = True)

In [ ]:
!pip3 install plotly

In [ ]:
import plotly.express as px

# Interactive Scatter Plot: Square Feet vs. Beds
scatter_fig = px.scatter(data, x='Square Feet', y='Beds', title='Square Feet vs. Beds')

# Box Plot: Distribution of Square Feet by Number of Bedrooms
box_fig = px.box(data, x='Beds', y='Square Feet', title='Distribution of Square Feet by Number of Bedrooms')


# Faceted Plot: Square Feet vs. Beds (Faceted by Number of Bathrooms)
faceted_fig = px.scatter(data, x='Beds', y='Square Feet', color='Baths', facet_col='Baths',
                         labels={'Beds': 'Number of Bedrooms', 'Square Feet': 'Square Feet', 'Baths': 'Number of Bathrooms'},
                         title='Square Feet vs. Beds (Faceted by Number of Bathrooms)')
# Violin Plot: Distribution of Square Feet
violin_fig = px.violin(data, y='Square Feet', title='Distribution of Square Feet')

# Display the plots
scatter_fig.show()
box_fig.show()
violin_fig.show()
faceted_fig.show()


In [ ]:
import cv2
import numpy as np

# Define the target image size
target_size = (64, 64)  # Example target size

# Initialize an empty list to store preprocessed images
preprocessed_images = []

# Loop through each image file path
for image_path in images:
    # Read the image
    image = cv2.imread(os.path.join(images_folder, image_path))

    # Resize the image
    resized_image = cv2.resize(image, target_size)

    # Normalize pixel values to range [0, 1]
    normalized_image = resized_image / 255.0

    # Convert the image to NumPy array and append to the list
    preprocessed_images.append(normalized_image)

# Convert the list of images to NumPy array
preprocessed_images = np.array(preprocessed_images)

# Display the shape of the preprocessed images array
print("Shape of preprocessed images array:", preprocessed_images.shape)


In [ ]:
!pip3 install optree

In [ ]:
!pip install --upgrade tensorflow keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Conv2D, Conv2DTranspose, BatchNormalization, LeakyReLU, Input, Concatenate
from tensorflow.keras.layers import Flatten, LeakyReLU, Input

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the dimensions of the random noise vector
latent_dim = 100

# Define the generator network
def build_generator():
    input_details = Input(shape=(4,))
    x = Dense(256)(input_details)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(64)(x)
    x = LeakyReLU(alpha=0.2)(x)
    output = Dense(64 * 64 * 3, activation='tanh')(x)
    output = Reshape((64, 64, 3))(output)
    model = Model(input_details, output)
    return model

# Define the discriminator network
def build_discriminator():
    input_image = Input(shape=(64, 64, 3))
    x = Conv2D(64, (3,3), strides=(2,2), padding='same')(input_image)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128, (3,3), strides=(2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(256, (3,3), strides=(2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(input_image, output)
    return model

# Define the combined GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    input_details = Input(shape=(4,))
    generated_image = generator(input_details)
    validity = discriminator(generated_image)
    model = Model(input_details, validity)
    return model

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

# Build the generator
generator = build_generator()

# Build and compile the combined GAN model
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

# Training loop
# Training loop
# Training loop
epochs = 100
batch_size = 64

for epoch in range(epochs):
    for batch_index in range(len(preprocessed_images) // batch_size):
        # Sample random noise vectors
        noise = np.random.randn(batch_size, 4)  # Ensure input shape matches generator input

        # Generate fake images
        generated_images = generator.predict(noise)

        # Train the discriminator
        real_images = preprocessed_images[batch_index * batch_size: (batch_index + 1) * batch_size]
        combined_images = np.concatenate([real_images, generated_images])
        labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
        labels += 0.05 * np.random.random(labels.shape)
        discriminator_loss = discriminator.train_on_batch(combined_images, labels)

        # Train the generator
        noise = np.random.randn(batch_size, 4)  # Ensure input shape matches generator input
        misleading_labels = np.ones((batch_size, 1))
        generator_loss = gan.train_on_batch(noise, misleading_labels)

    # Print progress
    print(f'Epoch {epoch+1}/{epochs}, Discriminator Loss: {discriminator_loss}, Generator Loss: {generator_loss}')

# Function to generate images based on input details
def generate_images(square_feet, beds, baths, garages):
    input_details = np.array([[square_feet, beds, baths, garages]])
    generated_image = generator.predict(input_details)
    generated_image = np.squeeze(generated_image, axis=0)  # Remove batch dimension
    generated_image = (generated_image * 0.5) + 0.5  # Rescale pixel values to [0, 1]
    return generated_image

# Example usage
new_image = generate_images(2500, 3, 2, 2)  # Provide input details
plt.imshow(new_image)
plt.axis('off')
plt.show()
